In [1]:
!pip3 install langchain --quiet
!pip3 install chromadb --quiet
!pip3 install sentence-transformers --quiet
!pip3 install unstructured --quiet

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import ChatGLM
from langchain.chains import RetrievalQA

In [3]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 导入文本
loader = UnstructuredFileLoader("demo.txt")
# 将文本转成 Document 目标
data = loader.load()
print(f'documents:{len(data)}')
# 初始化加载器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(data)
print("split_docs size:",len(split_docs))

documents:1
split_docs size: 36


In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import IPython
import sentence_transformers

In [5]:
embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec": "GanymedeNil/text2vec-large-chinese",
    "text2vec2":"uer/sbert-base-chinese-nli",
    "text2vec3":"shibing624/text2vec-base-chinese",
}
EMBEDDING_MODEL = "text2vec3"

In [6]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[EMBEDDING_MODEL], )

In [7]:
from langchain.vectorstores import Chroma

In [8]:
db = Chroma.from_documents(split_docs, embeddings,persist_directory="vector-result")

In [17]:
question = "恒大集团2022年负债总额有多少？"
similarDocs = db.similarity_search(question, include_metadata=True,k=4)
[print(x) for x in similarDocs]

page_content='巨亏之下，恒大汽车的财务状况可谓“捉襟见肘”。根据恒大汽车披露，截至2022年底其负债总额达到1838.72亿元，剔除1190.02亿元的住宅业务相关负债后，仍有648.70亿元的负债，其中包含流动' metadata={'source': 'demo.txt'}
page_content='巨亏之下，恒大汽车的财务状况可谓“捉襟见肘”。根据恒大汽车披露，截至2022年底其负债总额达到1838.72亿元，剔除1190.02亿元的住宅业务相关负债后，仍有648.70亿元的负债，其中包含流动' metadata={'source': 'demo.txt'}
page_content='巨亏之下，恒大汽车的财务状况可谓“捉襟见肘”。根据恒大汽车披露，截至2022年底其负债总额达到1838.72亿元，剔除1190.02亿元的住宅业务相关负债后，仍有648.70亿元的负债，其中包含流动' metadata={'source': 'demo.txt'}
page_content='巨亏之下，恒大汽车的财务状况可谓“捉襟见肘”。根据恒大汽车披露，截至2022年底其负债总额达到1838.72亿元，剔除1190.02亿元的住宅业务相关负债后，仍有648.70亿元的负债，其中包含流动' metadata={'source': 'demo.txt'}


[None, None, None, None]

## 接入ChatGLM做总结

In [10]:
endpoint_url = "http://127.0.0.1:8000"

# direct access endpoint in a proxied environment
# os.environ['NO_PROXY'] = '127.0.0.1'

llm = ChatGLM(
    endpoint_url=endpoint_url,
    max_token=80000,
    history=[],
    top_p=0.9,
    model_kwargs={"sample_model_args": False},
)

In [15]:
from langchain.chains import RetrievalQA
import IPython
retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
# 进行问答
query = "这篇文章谈论的是什么？"
print(qa.run(query))

这篇文章没有明确的主题或话题，它提供了几个与恒大集团相关的背景信息，包括一个有网友给恒大算了一笔账的故事，以及股票停牌窗口仅剩两月。这些信息似乎是用来报道或讨论恒大集团的财务状况或汽车销售业务，但文章没有提供更多上下文或明确的问题。


In [16]:
query = "恒大集团2022年负债总额有多少？"
print(qa.run(query))

根据恒大汽车披露，截至2022年底其负债总额达到1838.72亿元。


## 以上步骤可以直接用封装好的index_creator

In [ ]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)